In [1]:
import pandas as pd
from itertools import chain
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from wave_analisys import *

In [7]:
tests = pd.read_csv('data/Propagação de ondas/ultrasound_waves_data.csv')
tests = tests.sort_values(by=['sample_id', 'test_condition', 'wave_type', 'test_id'])
indexes = ['sample_id', 'test_condition', 'wave_type', 'test_id']
tests.set_index(indexes, drop=True, inplace=True)

to_array = lambda text: list(map(float, text.replace('[','').replace(']','').split(',')))
tests['sampling'] = tests['sampling'].apply(to_array)

# tests = filter_tests(tests, sample_id='MA01-02', test_condition='dry', wave_type='s')

In [8]:
threshold_factor = 0.5
times_to_pick= 5

time_cols = []
for t in range(times_to_pick):
    time_cols.append('time_%s' % str(t + 1))
    tests[time_cols[t]] = 0

In [9]:
for test in tests.itertuples():
    arrival_times = []
    
    threshold = max((map(abs, test.sampling))) * threshold_factor     
    arrival_times = zero_crossing_threshold(test, threshold, times_to_pick)
    
    [arrival_times.append(0) for i in range(times_to_pick - len(arrival_times))]
    
    tests.at[test.Index, time_cols] = arrival_times

In [10]:
std_factor = 1.5
mean_times = tests[time_cols].groupby(indexes[0:3]).agg(lambda x: mean_without_outliers(x, std_factor))

In [12]:
def ax_callback(test, cs, ax):

    factor = threshold_factor
    threshold = max((map(abs, test.sampling))) * factor
    ax.axhline(y=threshold, c='red', ls='--', label='Limiar: $\pm$%.2f' % threshold)
    ax.axhline(y=-threshold, c='red', ls='--')
        
    for i, col in enumerate(time_cols):
        time = getattr(test, col)
        if (time > 0):
            ax.plot(time, cs(time), 'o', c='red', ms=3, label='$t_%i$: %.2f$\mu$s' % (i + 1, time), zorder=30)

        mean_time = mean_times.loc[test.Index[0:3]][col]
        if mean_time > 0:
            ax.axvline(x=mean_time, c='green', label='Média %i: %.2f$\mu$s' % (i + 1, mean_time), zorder=10)
            
        handles, labels = ax.get_legend_handles_labels()
        labels.insert(1, '')
        handles.insert(1, mpatches.Patch(color='white'))
        
        ax.legend(handles=handles, labels=labels, prop={'size': 12}, frameon=True, edgecolor='black', ncol=6, loc=4)
        
def save_fig(index, tests_by_sample):
    plt.savefig('data/Propagação de ondas/Plots/%s_%s_%s.png' % index, bbox_inches='tight', dpi=300)

waveform_plot(tests, ax_callback=ax_callback, plot_callback=save_fig)

In [1]:
'''
p_waves = filter_tests(mean_times, wave_type='p').reset_index()
p_waves = p_waves.pivot(index='sample_id', columns='test_condition', values='time_1')
p_waves.columns = ['time_1_p_%s' % col_name for col_name in p_waves.columns.to_list()]

'''

s_waves = filter_tests(mean_times, wave_type='s').reset_index()
s_waves = s_waves.pivot(index='sample_id', columns='test_condition', values=time_cols)
s_waves.columns = ['%s_s_%s' % col_name for col_name in s_waves.columns.to_list()]

# results = pd.concat([p_waves, s_waves], axis=1)
results = s_waves
results = results.reindex(sorted(results.columns), axis=1)
display(results)

results.to_csv('data/Propagação de ondas/waves_arrival_times_0_66.csv', float_format='%.2f')

NameError: name 'filter_tests' is not defined